### Практическая работа к уроку №6

**Задание 1.**

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


**Задание 2.**

Обучите модель 2-ого уровня, при этом:

- Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар

- Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_ranker

- Вырос ли precision@5 при использовании двухуровневой модели?

#### Prepare data

In [105]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

In [8]:
data = pd.read_csv('../data/retail_train.csv')
item_features = pd.read_csv('../data/product.csv')
user_features = pd.read_csv('../data/hh_demographic.csv')

In [9]:
def print_stats_data(df_data, name_df):
    print(name_df)
    print(f"Shape: {df_data.shape} Users: {df_data[USER_COL].nunique()} Items: {df_data[ITEM_COL].nunique()}")

In [11]:
ITEM_COL = 'item_id'
USER_COL = 'user_id'

def prepare_data(VAL_MATCHER_WEEKS, VAL_RANKER_WEEKS):
    # column processing
    item_features.columns = [col.lower() for col in item_features.columns]
    user_features.columns = [col.lower() for col in user_features.columns]

    item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
    user_features.rename(columns={'household_key': USER_COL }, inplace=True)
    
    # берем данные для тренировки matching модели
    data_train_matcher = data[data['week_no'] < data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)]

    # берем данные для валидации matching модели
    data_val_matcher = data[(data['week_no'] >= data['week_no'].max() - (VAL_MATCHER_WEEKS + VAL_RANKER_WEEKS)) &
                          (data['week_no'] < data['week_no'].max() - (VAL_RANKER_WEEKS))]


    # берем данные для тренировки ranking модели
    data_train_ranker = data_val_matcher.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться

    # берем данные для теста ranking, matching модели
    data_val_ranker = data[data['week_no'] >= data['week_no'].max() - VAL_RANKER_WEEKS]
    
    print_stats_data(data_train_matcher,'train_matcher')
    print_stats_data(data_val_matcher,'val_matcher')
    print_stats_data(data_train_ranker,'train_ranker')
    print_stats_data(data_val_ranker,'val_ranker')
    
    # prefilter items: top-5000
    print('\n prefilter items in data_train_matcher:')
    n_items_before = data_train_matcher['item_id'].nunique()
    data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)
    n_items_after = data_train_matcher['item_id'].nunique()
    print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))
    
    # ищем общих пользователей
    print('\nMake cold-start to warm-start')
    common_users = data_train_matcher.user_id.values

    data_val_matcher = data_val_matcher[data_val_matcher.user_id.isin(common_users)]
    data_train_ranker = data_train_ranker[data_train_ranker.user_id.isin(common_users)]
    data_val_ranker = data_val_ranker[data_val_ranker.user_id.isin(common_users)]

    print_stats_data(data_train_matcher,'train_matcher')
    print_stats_data(data_val_matcher,'val_matcher')
    print_stats_data(data_train_ranker,'train_ranker')
    print_stats_data(data_val_ranker,'val_ranker')
    
    return data_train_matcher, data_val_matcher, data_train_ranker, data_val_ranker

In [12]:
VAL_MATCHER_WEEKS = 6
VAL_RANKER_WEEKS = 3

(data_train_matcher, data_val_matcher,
 data_train_ranker, data_val_ranker) = prepare_data(VAL_MATCHER_WEEKS, VAL_RANKER_WEEKS)

train_matcher
Shape: (2108779, 12) Users: 2498 Items: 83685
val_matcher
Shape: (169711, 12) Users: 2154 Items: 27649
train_ranker
Shape: (169711, 12) Users: 2154 Items: 27649
val_ranker
Shape: (118314, 12) Users: 2042 Items: 24329

 prefilter items in data_train_matcher:


/home/mayo889/Documents/GeekBrains/GeekBrains/recommendations/06_lesson/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 83685 to 5001

Make cold-start to warm-start
train_matcher
Shape: (861404, 13) Users: 2495 Items: 5001
val_matcher
Shape: (169615, 12) Users: 2151 Items: 27644
train_ranker
Shape: (169615, 12) Users: 2151 Items: 27644
val_ranker
Shape: (118282, 12) Users: 2040 Items: 24325


# Init/train recommender

In [16]:
recommender = MainRecommender(data_train_matcher)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

**Задание 1.**

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_matcher: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

In [106]:
ACTUAL_COL = 'actual'

result_eval_matcher = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [107]:
def calc_recall(df_data, col_name, top_k):
    return df_data.apply(lambda row: recall_at_k(row[col_name],
                                                 row[ACTUAL_COL], k=top_k), axis=1).mean()
        
def calc_precision(df_data, col_name, top_k):
    return df_data.apply(lambda row: precision_at_k(row[col_name],
                                                    row[ACTUAL_COL], k=top_k), axis=1).mean()

In [108]:
def base_predict(df_result, target_col_name, result_col_name, recommend_model, TOPK_RECALL, TOPK_PRECISION):
    time_start = time.time()
    df_result[result_col_name] = df_result[target_col_name].apply(lambda x: recommend_model(x, N=TOPK_RECALL))
    predict_time = time.time() - time_start
    
    # Сразу подсчитываем recall_at_k и precision_at_k (для сравнения со 2 уровнем модели)
    recall = calc_recall(df_result, result_col_name, TOPK_RECALL)
    precision = calc_recall(df_result, result_col_name, TOPK_PRECISION)  
    
    return df_result, recall, precision, predict_time

In [109]:
all_recommend_models = {
    'own_rec': recommender.get_own_recommendations,
    'sim_item_rec': recommender.get_similar_items_recommendation,
    'als_rec': recommender.get_als_recommendations
}

TOPK_RECALL = 50
TOPK_PRECISION = 5
result_dct = {}

for result_col_name, recommend_model in all_recommend_models.items():
    print(f"Processing {result_col_name}...")
    result_eval_matcher, recall, precision, predict_time = base_predict(result_eval_matcher,
                                                                                  USER_COL,
                                                                                  result_col_name,
                                                                                  recommend_model,
                                                                                  TOPK_RECALL,
                                                                                  TOPK_PRECISION)
    result_dct[result_col_name] = [recall, precision, predict_time]

Processing own_rec...
Processing sim_item_rec...
Processing als_rec...


In [113]:
result_train_recommender = pd.DataFrame.from_dict(result_dct, orient='index',
                                                 columns=['recall@50', 'precision@5', 'time, sec'])
result_train_recommender.sort_values(by='recall@50', ascending=False, inplace=True)
result_train_recommender

,recall@50,precision@5,"time, sec"
own_rec,0.065257,0.018202,9.390431
als_rec,0.047518,0.012348,9.644044
sim_item_rec,0.032989,0.005286,19.389914


own recommendtions + top-popular лучший recall

**Задание 1.**

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  

In [114]:
result_eval_matcher_2 = data_val_matcher.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_matcher_2.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher_2.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [115]:
TOPK_RECALL_LIST = [20, 50, 100, 200, 500]
recommend_model = recommender.get_own_recommendations
result_dct = {}

for TOPK_RECALL in TOPK_RECALL_LIST:
    result_col_name = f"own_rec_k={TOPK_RECALL}"
    print(f"Processing {result_col_name}...")
    
    result_eval_matcher_2, recall, precision, predict_time = base_predict(result_eval_matcher_2,
                                                                          USER_COL,
                                                                          result_col_name,
                                                                          recommend_model,
                                                                          TOPK_RECALL,
                                                                          TOPK_PRECISION)
    result_dct[result_col_name] = [recall, precision, predict_time]

Processing own_rec_k=20...
Processing own_rec_k=50...
Processing own_rec_k=100...
Processing own_rec_k=200...
Processing own_rec_k=500...


In [117]:
result_own_rec = pd.DataFrame.from_dict(result_dct, orient='index',
                                        columns=['recall@k', 'precision@5', 'time, sec'])
result_own_rec.sort_values(by='recall@k', ascending=False, inplace=True)
result_own_rec

,recall@k,precision@5,"time, sec"
own_rec_k=500,0.182053,0.018202,9.739938
own_rec_k=200,0.135373,0.018202,10.896492
own_rec_k=100,0.096045,0.018202,9.790130
own_rec_k=50,0.065257,0.018202,9.297297
own_rec_k=20,0.039284,0.018202,9.018145


C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?

При росте k увеличивается recall@k, но повышать k бесконечно нельзя, так как чем больше k, тем дольше нам придется обучать модель 2 уровня и большие вычислительные мощности нам придется использовать.

В нашем случае вполне можно остановиться на значении k=500, так как разница в recall существенна

### Ranking part
### Обучение модели 2 уровня

In [471]:
def prepare_rank_dataset(N_PREDICT):
    # Negativ examples with own recommendations
    
    # взяли пользователей из трейна для ранжирования
    df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
    df_match_candidates.columns = [USER_COL]
    
    # собираем кандитатов с первого этапа (matcher)
    df_match_candidates['candidates'] = df_match_candidates[USER_COL]\
                        .apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))
    
    df_items = df_match_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1)\
                                  .stack().reset_index(level=1, drop=True)
    df_items.name = 'item_id'
    df_match_candidates = df_match_candidates.drop('candidates', axis=1).join(df_items)
    
    print_stats_data(df_match_candidates, 'Negativ examples')
    
    # Positive examples with real purchases
    df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
    df_ranker_train['target'] = 1  # тут только покупки 
    
    print_stats_data(df_ranker_train, 'Positive examples')
    
    # Merge positive and negative examples
    df_ranker_train = df_match_candidates.merge(df_ranker_train, on=[USER_COL, ITEM_COL], how='left')

    # чистим дубликаты
    df_ranker_train = df_ranker_train.drop_duplicates(subset=[USER_COL, ITEM_COL])
    df_ranker_train['target'].fillna(0, inplace= True)
    print_stats_data(df_ranker_train, 'Rank dataset')
    
    return df_ranker_train

In [472]:
N_PREDICT = 100
df_ranker_train = prepare_rank_dataset(N_PREDICT)
df_ranker_train.head()

Negativ examples
Shape: (107550, 2) Users: 2151 Items: 4574
Positive examples
Shape: (169615, 3) Users: 2151 Items: 27644
Rank dataset
Shape: (106972, 3) Users: 2151 Items: 4574


,user_id,item_id,target
0,2070,1105426,0.0
1,2070,1097350,0.0
2,2070,879194,0.0
3,2070,948640,0.0
4,2070,928263,0.0


### Генерация новых фичей

In [473]:
# Соберем всю информацию, которую мы знаем о покупках, для генерации новых фичей
features_train_ranker = data_train_ranker.copy()

features_train_ranker = features_train_ranker.merge(item_features, on='item_id', how='left')
features_train_ranker = features_train_ranker.merge(user_features, on='user_id', how='left')

features_train_ranker.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,40618492260,594,1019940,1,1.00,311,-0.29,40,86,...,SOFT DRINKS,SOFT DRINK BOTTLE NON-CARB (EX,20 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2021,40618753059,594,840361,1,0.99,443,0.00,101,86,...,EGGS,EGGS - LARGE,1 DZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,40618753059,594,856060,1,1.77,443,-0.09,101,86,...,CANNED JUICES,FRUIT DRINKS: CANNED & GLASS (,128 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,40618753059,594,869344,1,1.67,443,-0.22,101,86,...,FRZN VEGETABLE/VEG DSH,FRZN BAGGED VEGETABLES - PLAIN,16 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,40618753059,594,896862,2,5.00,443,-2.98,101,86,...,BACON,ECONOMY,1 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [345]:
# features_train_ranker.columns.tolist()

#### Генерация фичей для user_id

##### Средний чек

In [474]:
add_user_features = pd.DataFrame(features_train_ranker.groupby('user_id')['sales_value'].median()).reset_index()
add_user_features.rename(columns={'sales_value': 'avg_sales_value'}, inplace=True)

add_user_features.head()

,user_id,avg_sales_value
0,1,2.290
1,2,2.000
2,4,3.315
3,6,2.495
4,7,1.595


##### Средняя сумма покупки 1 товара в каждой категории

In [475]:
# Уверен, что формирую фичу не рационально, хотелось бы конечно иначе

# Сначала найдем все категории и заполним для каждого юзера среднюю сумму покупки нулями,
# так как пользователь мог не совершать покупку в данной категории
departments = features_train_ranker['department'].unique().tolist()
departments.remove(' ')
for department in departments:
    add_user_features[f'avg_buy_{department}'] = 0
add_user_features.head()

,user_id,avg_sales_value,avg_buy_GROCERY,avg_buy_MEAT-PCKGD,avg_buy_MEAT,avg_buy_DRUG GM,avg_buy_KIOSK-GAS,avg_buy_PRODUCE,avg_buy_NUTRITION,avg_buy_SEAFOOD,...,avg_buy_COUP/STR & MFG,avg_buy_RX,avg_buy_DELI/SNACK BAR,avg_buy_RESTAURANT,avg_buy_FROZEN GROCERY,avg_buy_GM MERCH EXP,avg_buy_AUTOMOTIVE,avg_buy_CNTRL/STORE SUP,avg_buy_GARDEN CENTER,avg_buy_PHOTO
0,1,2.290,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2.000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,3.315,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,2.495,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,1.595,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [476]:
# Найдем для каждого пользователя среднюю сумму покупки в каждой категории
user_department_price = pd.DataFrame(features_train_ranker.groupby(['user_id', 'department'])['sales_value'].median()).reset_index()
user_department_price = user_department_price[user_department_price['department'] != ' ']
user_department_price.head()

,user_id,department,sales_value
1,1,DELI,2.395
2,1,DRUG GM,2.500
3,1,GROCERY,2.290
4,1,MEAT-PCKGD,2.815
5,1,NUTRITION,2.690


In [477]:
# Теперь пройдемся по каждому среднему чеку в категории и заполним основную таблицу с фичами
for user_id, department, avg_buy in user_department_price.values:
    add_user_features.loc[add_user_features['user_id'] == user_id, 
                         f"avg_buy_{department}"] = avg_buy

In [478]:
add_user_features.head()

,user_id,avg_sales_value,avg_buy_GROCERY,avg_buy_MEAT-PCKGD,avg_buy_MEAT,avg_buy_DRUG GM,avg_buy_KIOSK-GAS,avg_buy_PRODUCE,avg_buy_NUTRITION,avg_buy_SEAFOOD,...,avg_buy_COUP/STR & MFG,avg_buy_RX,avg_buy_DELI/SNACK BAR,avg_buy_RESTAURANT,avg_buy_FROZEN GROCERY,avg_buy_GM MERCH EXP,avg_buy_AUTOMOTIVE,avg_buy_CNTRL/STORE SUP,avg_buy_GARDEN CENTER,avg_buy_PHOTO
0,1,2.290,2.29,2.815,0.00,2.500,0.0,1.315,2.69,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
1,2,2.000,1.65,2.500,4.59,3.170,0.0,1.350,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
2,4,3.315,3.34,4.145,0.00,2.640,0.0,0.000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
3,6,2.495,2.22,0.000,3.52,2.190,0.0,2.495,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
4,7,1.595,1.50,2.500,5.50,2.035,0.0,2.490,2.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0


#### Генерация фичей для item_id

##### Среднее кол-во покупок в неделю

In [479]:
# Сначала просуммируем количество покупок товара по каждой неделе
item_week_quantity = pd.DataFrame(features_train_ranker.groupby(['item_id', 'week_no'])['quantity'].sum()).reset_index()
item_week_quantity.head()

,item_id,week_no,quantity
0,28116,91,1
1,28117,91,1
2,28143,91,1
3,28186,90,1
4,28304,91,1


In [480]:
# Теперь найдем среднее количество покупок по неделям для каждого товара
add_item_features = pd.DataFrame(item_week_quantity.groupby('item_id')['quantity'].mean()).reset_index()
add_item_features.rename(columns={'quantity': 'quantity_per_week'}, inplace=True)

add_item_features.head()

,item_id,quantity_per_week
0,28116,1.0
1,28117,1.0
2,28143,1.0
3,28186,1.0
4,28304,1.0


##### Цена товара

In [481]:
# Найдем как sales_value / quantity
features_train_ranker['price'] = features_train_ranker['sales_value'] / features_train_ranker['quantity']

# У товара может меняться цена во времени,
# поэтому усредним полученные значения цены по каждому товару
item_prices = pd.DataFrame(features_train_ranker.groupby('item_id')['price'].mean()).reset_index()

# Смержем с уже созданной фичей для товаров
add_item_features = add_item_features.merge(item_prices, on='item_id', how='left')
add_item_features.head()

,item_id,quantity_per_week,price
0,28116,1.0,0.33
1,28117,1.0,0.34
2,28143,1.0,0.33
3,28186,1.0,0.79
4,28304,1.0,0.53


#### Генерация фичей user_id - item_id

In [482]:
"""
В начале уже соединим все фичи, которые имеем

из item_features и user_features возьмем не все
"""
item_columns = ['item_id', 'department', 'brand']
df_ranker_train = df_ranker_train.merge(item_features[item_columns], on='item_id', how='left')
df_ranker_train = df_ranker_train.merge(add_item_features, on='item_id', how='left')

user_columns = ['user_id', 'age_desc', 'marital_status_code', 'income_desc', 'household_size_desc']
df_ranker_train = df_ranker_train.merge(user_features, on='user_id', how='left')
df_ranker_train = df_ranker_train.merge(add_user_features, on='user_id', how='left')

df_ranker_train.head(2)

,user_id,item_id,target,department,brand,quantity_per_week,price,age_desc,marital_status_code,income_desc,...,avg_buy_COUP/STR & MFG,avg_buy_RX,avg_buy_DELI/SNACK BAR,avg_buy_RESTAURANT,avg_buy_FROZEN GROCERY,avg_buy_GM MERCH EXP,avg_buy_AUTOMOTIVE,avg_buy_CNTRL/STORE SUP,avg_buy_GARDEN CENTER,avg_buy_PHOTO
0,2070,1105426,0.0,DELI,Private,1.666667,3.99,45-54,U,50-74K,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
1,2070,1097350,0.0,GROCERY,National,1.000000,10.99,45-54,U,50-74K,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0


#### (Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)) - (Цена item_id)

In [483]:
# Найдем среднюю цену покупки по каждой категории
avg_price_in_department = pd.DataFrame(features_train_ranker.groupby('department')['sales_value'].mean()).reset_index()

avg_price_in_department = dict(zip(avg_price_in_department.department.tolist(),
                                   avg_price_in_department.sales_value.tolist()))
avg_price_in_department

{' ': 0.0,
 'AUTOMOTIVE': 5.39,
 'CHEF SHOPPE': 2.883174603174604,
 'CNTRL/STORE SUP': 1.125,
 'COSMETICS': 4.466973947895789,
 'COUP/STR & MFG': 1.5757894736842104,
 'DELI': 4.527462919594057,
 'DELI/SNACK BAR': 1.64,
 'DRUG GM': 3.977360863697661,
 'FLORAL': 9.130999999999982,
 'FROZEN GROCERY': 3.949166666666667,
 'GARDEN CENTER': 21.75909090909091,
 'GM MERCH EXP': 1.36,
 'GROCERY': 2.5319687953556853,
 'KIOSK-GAS': 23.284805369127522,
 'MEAT': 6.485358877495984,
 'MEAT-PCKGD': 3.834287925696735,
 'MISC SALES TRAN': 22.355680751173693,
 'MISC. TRANS.': 3.4740099009901018,
 'NUTRITION': 3.2860244026435708,
 'PASTRY': 3.4611332007951505,
 'PHOTO': 2.49,
 'PRODUCE': 2.314030509835602,
 'RESTAURANT': 5.848500000000001,
 'RX': 13.440000000000001,
 'SALAD BAR': 3.169469453376207,
 'SEAFOOD': 7.0232217573221805,
 'SEAFOOD-PCKGD': 5.7545234493191835,
 'SPIRITS': 9.52620000000001,
 'TRAVEL & LEISUR': 3.101451612903225,
 'VIDEO': 12.79}

In [484]:
# Пройдемся по основному набору данных  и посчитаем разность
# средней суммы покупки для данной категории товара и стоимостью самого товара

idx_department = df_ranker_train.columns.get_loc("department")
idx_item_price = df_ranker_train.columns.get_loc("price")

df_ranker_train['hard_feature_1'] = df_ranker_train[['department', 'price']]\
        .apply(lambda row: avg_price_in_department[row[0]] - row[1], axis=1)

df_ranker_train.head(2)

,user_id,item_id,target,department,brand,quantity_per_week,price,age_desc,marital_status_code,income_desc,...,avg_buy_RX,avg_buy_DELI/SNACK BAR,avg_buy_RESTAURANT,avg_buy_FROZEN GROCERY,avg_buy_GM MERCH EXP,avg_buy_AUTOMOTIVE,avg_buy_CNTRL/STORE SUP,avg_buy_GARDEN CENTER,avg_buy_PHOTO,hard_feature_1
0,2070,1105426,0.0,DELI,Private,1.666667,3.99,45-54,U,50-74K,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.537463
1,2070,1097350,0.0,GROCERY,National,1.000000,10.99,45-54,U,50-74K,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,-8.458031


##### (Кол-во покупок юзером конкретной категории в неделю) - (Среднее кол-во покупок всеми юзерами конкретной категории в неделю)

In [485]:
# Сначала посчитаем сколько покупок сделал каждый юзер в каждой категории
user_department_number_purchases = pd.DataFrame(df_ranker_train.groupby(['user_id', 'department'])['item_id'].count()).reset_index()
user_department_number_purchases.rename(columns={'item_id': 'number_purchases'}, inplace=True)
user_department_number_purchases.head(2)

,user_id,department,number_purchases
0,1,DELI,3
1,1,DRUG GM,4


In [486]:
# Теперь усредним количество покупок по каждой категории
avg_purchases_per_department = dict(user_department_number_purchases.groupby('department')['number_purchases'].mean())
avg_purchases_per_department

{'COSMETICS': 1.0,
 'DELI': 2.850783699059561,
 'DRUG GM': 3.846903949293028,
 'FLORAL': 1.2804878048780488,
 'GROCERY': 30.95676429567643,
 'MEAT': 3.9530546623794214,
 'MEAT-PCKGD': 4.4608478802992515,
 'MISC. TRANS.': 1.0,
 'NUTRITION': 1.926489226869455,
 'PASTRY': 1.6715686274509804,
 'PRODUCE': 4.913738019169329,
 'SEAFOOD': 1.3478260869565217,
 'SEAFOOD-PCKGD': 1.4834054834054835,
 'SPIRITS': 1.0620689655172413}

In [487]:
user_department_number_purchases['hard_feature_2'] = user_department_number_purchases.apply(lambda row: row[2] - avg_purchases_per_department[row[1]],
                                                                                            axis=1)

# Удалим колонку перед мержем
user_department_number_purchases.drop('number_purchases', axis=1, inplace=True)

user_department_number_purchases.head(2)

,user_id,department,hard_feature_2
0,1,DELI,0.149216
1,1,DRUG GM,0.153096


In [488]:
# Смержим данную фичу к основному датасету

df_ranker_train= df_ranker_train.merge(user_department_number_purchases,
                                       on=['user_id', 'department'], how='left')
df_ranker_train.head()

,user_id,item_id,target,department,brand,quantity_per_week,price,age_desc,marital_status_code,income_desc,...,avg_buy_DELI/SNACK BAR,avg_buy_RESTAURANT,avg_buy_FROZEN GROCERY,avg_buy_GM MERCH EXP,avg_buy_AUTOMOTIVE,avg_buy_CNTRL/STORE SUP,avg_buy_GARDEN CENTER,avg_buy_PHOTO,hard_feature_1,hard_feature_2
0,2070,1105426,0.0,DELI,Private,1.666667,3.99,45-54,U,50-74K,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.537463,1.149216
1,2070,1097350,0.0,GROCERY,National,1.000000,10.99,45-54,U,50-74K,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,-8.458031,3.043236
2,2070,879194,0.0,DRUG GM,Private,NaN,NaN,45-54,U,50-74K,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,2.153096
3,2070,948640,0.0,DRUG GM,National,NaN,NaN,45-54,U,50-74K,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,NaN,2.153096
4,2070,928263,0.0,DRUG GM,Private,2.200000,7.99,45-54,U,50-74K,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,-4.012639,2.153096


### Обучение модели ранжирования

In [490]:
X_train = df_ranker_train.drop('target', axis=1)
y_train = df_ranker_train[['target']]

In [501]:
cat_feats = [column for column in df_ranker_train.columns
                    if df_ranker_train[column].dtypes == 'object']
X_train[cat_feats] = X_train[cat_feats].astype('category')
cat_feats

['department',
 'brand',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [502]:
lgb = LGBMClassifier(objective='binary',
                     max_depth=8,
                     n_estimators=300,
                     learning_rate=0.05,
                     categorical_column=cat_feats)

lgb.fit(X_train, y_train)

/home/mayo889/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/mayo889/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/mayo889/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


LGBMClassifier(categorical_column=['department', 'brand', 'age_desc',
                                   'marital_status_code', 'income_desc',
                                   'homeowner_desc', 'hh_comp_desc',
                                   'household_size_desc', 'kid_category_desc'],
               learning_rate=0.05, max_depth=8, n_estimators=300,
               objective='binary')

In [504]:
train_preds = lgb.predict_proba(X_train)
df_ranker_predict = df_ranker_train.copy()
df_ranker_predict['proba_item_purchase'] = train_preds[:,1]

- Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_ranker

- Вырос ли precision@5 при использовании двухуровневой модели?

In [505]:
result_eval_ranker = data_val_ranker.groupby(USER_COL)[ITEM_COL].unique().reset_index()
result_eval_ranker.columns=[USER_COL, ACTUAL_COL]
result_eval_ranker.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [506]:
%%time
result_eval_ranker['own_rec'] = result_eval_ranker[USER_COL].apply(lambda x: recommender.get_own_recommendations(x, N=N_PREDICT))

CPU times: user 9.19 s, sys: 0 ns, total: 9.19 s
Wall time: 9.21 s


In [508]:
calc_precision(result_eval_ranker, 'own_rec', TOPK_PRECISION)

0.1444117647058813

In [509]:
def rerank(user_id):
    return df_ranker_predict[df_ranker_predict[USER_COL]==user_id]\
                .sort_values('proba_item_purchase', ascending=False).head(5).item_id.tolist()

In [510]:
result_eval_ranker['reranked_own_rec'] = result_eval_ranker[USER_COL].apply(lambda user_id: rerank(user_id))

In [511]:
calc_precision(result_eval_ranker, 'reranked_own_rec', TOPK_PRECISION)

/home/mayo889/Documents/GeekBrains/GeekBrains/recommendations/06_lesson/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


0.15738903394255754

Переранжирование дало нам улучшение на 9%